In [11]:
from bs4 import BeautifulSoup
import lxml #is a parser for HTMl
#import webscraping
import requests
import os
import re
import csv
#from selenium import webdriver
from langdetect import detect

We shoul generalize these functions... in webscraping.py!

## **[RQ1.1] Get the list of books**

We start to scrape the three pages to extract the URLs for each of them

In [12]:
UrlsFiles = open("urlpages.txt", "w")

url = "https://www.goodreads.com/list/show/1.Best_Books_Ever?page="+str(2)
page = requests.get(url)
soup = BeautifulSoup(page.content, features='lxml')
for a in soup.find_all('a', class_="bookTitle"): #.contents[0].split('\n')[1].strip()
    UrlsFiles.write(a.get('href')+'\n')

#UrlsFiles.truncate(UrlsFiles.tell()-1) see how to remove the '\n'
UrlsFiles.close()

## **RQ[1.2] Crawl books**

In [13]:
headpart = "https://www.goodreads.com"

In [14]:
direct = "htmlpages" #Directory  
parent_dir = "./" #Parent Directory path 
pathAncestor = os.path.join(parent_dir, direct) #Path
os.mkdir(pathAncestor) #create the folder in the path

In [15]:
for i in range(1,101):
    os.makedirs(os.path.join(pathAncestor, 'page ' + str(i)))

In [16]:
UrlsFiles = open("urlpages.txt", "r")

for i, x in enumerate(UrlsFiles,1):
    subdirectory = pathAncestor + "/page " + str(i)
    article_name = "/article_"+str(i)+".html"
    
    complete_path = subdirectory + article_name
    with open(complete_path, "wb") as ip_file:
        link = headpart + x
        page = requests.get(link)
        
        soup = BeautifulSoup(page.text, features='lxml')
        
        ip_file.write(soup.encode('utf-8'))
        ip_file.close()

UrlsFiles.close()

## **RQ[1.3] Parse downloaded pages**

In [17]:
def scrap_book(href, tsv_writer, Url):
    r = requests.get(href) #call to have the html page with the useful information for my analysis
    soup = BeautifulSoup(r.text, features="lxml") #parse the text
    
    #extract rating and review count
    ratings = soup.find_all('a', href="#other_reviews") #search the ratings in its
    rating_count = -1
    rating = -1
    for raiting in ratings:
        if raiting.find_all('meta', itemprop="ratingCount"):
            ratingCount = raiting.text.replace('\n', '').strip().split(' ')[0].replace(',', '')
        elif raiting.find_all('meta', itemprop="reviewCount"):
            reviewCount = raiting.text.replace('\n', '').strip().split(' ')[0].replace(',', '')
    
    #extract the book title
    bookTitle = soup.find_all('h1')[0].contents[0].replace('\n', '').strip()

    #extract the book authors
    bookAuthors = soup.find_all('span', itemprop='name')[0].contents[0]
    
    #extract the book authors, we shoul FIX it.
    try:
        Plot = soup.find_all('div', id="description")[0].contents[3].text
    except:
        Plot = soup.find_all('div', id="description")[0].contents[1].text
    if detect(Plot) != "en":
        Plot = "NaN"
    
    #extract the date
    date = soup.find_all('div', id="details")[0].contents[3].text.replace('\n', '').strip().split()
    Published = date[1]+" "+date[2]+" "+date[3]
    
    #Rating Value
    ratingValue = soup.find('span', itemprop="ratingValue").text.strip()
    
    #Number of pages
    NumberofPages = soup.find('span', itemprop="numberOfPages").text.split()[0]
    
    #Title series
    bookSeries = soup.find_all('a', href= re.compile(r'/series/*'))
    if not bookSeries:
        bookSeries = "NaN"
    else:
        bookSeries = soup.find_all('a', href= re.compile(r'/series/*'))[0].contents[0].strip()
    
    #Places
    Setting = []
    for places in soup.find_all('a', href= re.compile(r'/places/*')):
        Setting.append(places.text)
    Setting = ", ".join(Setting) if len(Setting)>=1 else "NaN"
        
    #list of characters
    Characters = []
    for character in soup.find_all('a', href= re.compile(r'/characters/*') ):
        Characters.append(character.text)
    Characters = ", ".join(Characters) if len(Characters)>=1 else "NaN"
    
    tsv_writer.writerow([bookTitle, bookSeries, bookAuthors, ratingValue, ratingCount, reviewCount, Plot, NumberofPages, Published, Characters, Setting, Url])

In [18]:
UrlsFiles = open("urlpages.txt", "r")

with open('./output.tsv', 'w', encoding="utf-8", newline='') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    tsv_writer.writerow(['bookTitle', 'bookSeries', 'bookAuthors', 'ratingValue', 
                    'ratingCount', 'reviewCount', 'Plot', 'NumberofPages', 'Published',
                    'Characters', 'Setting', 'Url'])
    for link in UrlsFiles:
        scrap_book("https://www.goodreads.com" + link, tsv_writer, link)

UrlsFiles.close()

In [121]:
#DEF Cleaning
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer 
final_plot=[]
stemmer=PorterStemmer()
stop = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer() 
df=pd.read_csv('output.tsv',sep="\t")
for i in range(0,len(df)):
    new_plot=[]
    plot=df.loc[i,"Plot"]
    tokenizer = RegexpTokenizer("[\w']+")  
    plot = tokenizer.tokenize(plot.lower()) 
    for word in plot:
        if word not in stop:
            new_plot.append(word) 
    final = [lemmatizer.lemmatize(i) for i in new_plot]
    final1 = [stemmer.stem(word) for word in final]
    final_plot.append(final1)

#DEF Vocabulary 
vocabulary={}
i = 0
for element in final_plot:
    for sub_element in element:
        if(i>0 and sub_element in vocabulary):
            continue
        else:
            i= i+1
            vocabulary[sub_element] = i

print(vocabulary)

{'horribl': 1, 'textur': 2, 'fabric': 3, 'woven': 4, "ships'": 5, 'cabl': 6, 'hawser': 7, 'polar': 8, 'wind': 9, 'blow': 10, 'bird': 11, 'prey': 12, 'hover': 13, 'melvil': 14, 'wrote': 15, 'masterpiec': 16, 'one': 17, 'greatest': 18, 'work': 19, 'imagin': 20, 'literari': 21, 'histori': 22, 'part': 23, 'mobi': 24, 'dick': 25, 'stori': 26, 'eerili': 27, 'compel': 28, 'madman': 29, 'pursu': 30, 'unholi': 31, 'war': 32, 'creatur': 33, 'vast': 34, 'danger': 35, 'unknow': 36, 'sea': 37, 'novel': 38, 'adventur': 39, 'encyclopaedia': 40, 'whale': 41, 'lore': 42, 'legend': 43, 'book': 44, 'seen': 45, "author'": 46, 'lifelong': 47, 'medit': 48, 'america': 49, 'written': 50, 'wonder': 51, 'redempt': 52, 'humour': 53, 'also': 54, 'profound': 55, 'inquiri': 56, 'charact': 57, 'faith': 58, 'natur': 59, 'percept': 60, 'edit': 61, 'reproduc': 62, 'definit': 63, 'text': 64, 'includ': 65, 'invalu': 66, 'explanatori': 67, 'note': 68, 'along': 69, 'map': 70, 'illustr': 71, 'glossari': 72, 'nautic': 73, 't